Group Project Weather API Call

In [ ]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import HTML, display

In [ ]:
load_dotenv()  # Load environment variables from .env file
TICKETMASTER_API_KEY = os.getenv('TICKETMASTER_API_KEY')
TOMORROW_API_KEY = os.getenv('TOMORROOW_API_KEY')   

TICKETMASTER_Events_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/events.json"
TICKETMASTER_Venues_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/venues/.json"
TOMORROW_API_Endpoint =  "https://api.tomorrow.io/v4/weather/forecast?location=32.71571,-117.16472"

In [ ]:
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

conn = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE
)

In [ ]:
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Get the date 7 days from today
next_7_days = today + timedelta(days=6)

# Format the dates in the correct format (e.g., 'YYYY-MM-DD')
start_date = today.strftime('%Y-%m-%dT00:00:00Z')
end_date = next_7_days.strftime('%Y-%m-%dT23:59:59Z')

print("Start Date:", start_date)
print("End Date:", end_date)

In [ ]:
# Weather API
# API pull with parameters
params = {
    "apikey": TOMORROW_API_KEY,       
    "city": "San Diego", 
    "startDateTime": start_date,  # Start date (today)
    "endDateTime": end_date      # End date (7 days later)                
}

# Send the request to the API
response = requests.get(TOMORROW_API_Endpoint, params=params)

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    print("API call succeeded!")
    data = response.json()  # The API response in JSON format
    print(data)             # Print the data (you'll see the events info here)
else:
    print(f"API call failed with status code: {response.status_code}")
    print("Response details:", response.text)  # Print more details on the failure

In [17]:
# Extract temperature, humidity, rain, and weather code for each hour
weather_data = []

# Base URL for the weather icons from the GitHub repository
base_url = "https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/"

# Dictionary to map weather codes to icon URLs
weather_code_icons = {
    1100: f'{base_url}mostly_clear_day.png',  # Example: mostly clear day
    1101: f'{base_url}partly_cloudy.png',  # Example: partly cloudy
    1102: f'{base_url}mostly_cloudy.png',  # Example: mostly cloudy
    1001: f'{base_url}cloudy.png',  # Example: cloudy
    2100: f'{base_url}fog_light.png',  # Example: light fog
    2000: f'{base_url}fog.png',  # Example: fog
    4000: f'{base_url}drizzle.png',  # Example: drizzle
    4200: f'{base_url}rain_light.png',  # Example: light rain
    4001: f'{base_url}rain.png',  # Example: rain
    4201: f'{base_url}rain_heavy.png',  # Example: heavy rain
    6000: f'{base_url}freezing_drizzle.png',  # Example: freezing drizzle
    6200: f'{base_url}freezing_rain_light.png',  # Example: light freezing rain
    6001: f'{base_url}freezing_rain.png',  # Example: freezing rain
    6201: f'{base_url}freezing_rain_heavy.png',  # Example: heavy freezing rain
    8000: f'{base_url}tstorm.png',  # Example: thunderstorm
}


for interval in data['timelines']['hourly']:
    time = interval['time']
    temperature = interval['values']['temperature']
    temperatureApparent = interval['values']['temperatureApparent']
    humidity = interval['values']['humidity']
    rain = interval['values']['rainIntensity']
    weather_codeday = interval['values']['weatherCode']
  
    # Get the icon URL for the weather code
    weather_icon = weather_code_icons.get(weather_codeday, f'{base_url}default.png')  # Use 'default.png' if code not found
    
    weather_data.append({
        'time': time,
        'temperature': temperature,
        'temperatureApparent': temperatureApparent,
        'humidity': humidity,
        'rain': rain,
        'weather_code': weather_codeday,
        'weather_icon': weather_code_icons
    })

 

# Convert values to a DataFrame for easier visualization
weather_df = pd.DataFrame(weather_data)
print(weather_df)


# Function to display icons in the DataFrame
def path_to_image_html(path):
    return f'<img src="{path}" width="50" height="50">'

# Apply the function to the 'weather_icon' column
weather_df['weather_icon'] = weather_df['weather_icon'].apply(path_to_image_html)

# Display the DataFrame with icons
display(HTML(weather_df.to_html(escape=False)))

                     time  temperature  temperatureApparent  humidity  rain  \
0    2025-02-11T00:00:00Z         14.0                 14.0        63   0.0   
1    2025-02-11T01:00:00Z         13.9                 13.9        64   0.0   
2    2025-02-11T02:00:00Z         13.5                 13.5        64   0.0   
3    2025-02-11T03:00:00Z         13.2                 13.2        64   0.0   
4    2025-02-11T04:00:00Z         13.1                 13.1        65   0.0   
..                    ...          ...                  ...       ...   ...   
115  2025-02-15T19:00:00Z         14.0                 14.0        72   0.0   
116  2025-02-15T20:00:00Z         14.2                 14.2        73   0.0   
117  2025-02-15T21:00:00Z         14.4                 14.4        74   0.0   
118  2025-02-15T22:00:00Z         14.2                 14.2        75   0.0   
119  2025-02-15T23:00:00Z         14.1                 14.1        77   0.0   

     weather_code                                  

,time,temperature,temperatureApparent,humidity,rain,weather_code,weather_icon
0,2025-02-11T00:00:00Z,14.0,14.0,63,0.00,1001,"<img src=""{1100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_clear_day.png', 1101: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/partly_cloudy.png', 1102: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_cloudy.png', 1001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/cloudy.png', 2100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog_light.png', 2000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog.png', 4000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/drizzle.png', 4200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_light.png', 4001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain.png', 4201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_heavy.png', 6000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_drizzle.png', 6200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_light.png', 6001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain.png', 6201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_heavy.png', 8000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/tstorm.png'}"" width=""50"" height=""50"">"
1,2025-02-11T01:00:00Z,13.9,13.9,64,0.00,1001,"<img src=""{1100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_clear_day.png', 1101: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/partly_cloudy.png', 1102: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_cloudy.png', 1001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/cloudy.png', 2100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog_light.png', 2000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog.png', 4000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/drizzle.png', 4200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_light.png', 4001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain.png', 4201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_heavy.png', 6000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_drizzle.png', 6200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_light.png', 6001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain.png', 6201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_heavy.png', 8000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/tstorm.png'}"" width=""50"" height=""50"">"
2,2025-02-11T02:00:00Z,13.5,13.5,64,0.00,1001,"<img src=""{1100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_clear_day.png', 1101: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/partly_cloudy.png', 1102: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_cloudy.png', 1001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes